<a href="https://colab.research.google.com/github/wakame1367/Notebooks/blob/master/sentinel2_cloud_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q s2cloudless rasterio pystac-client stackstac folium geojson shapely

In [15]:
# 東京
bbox = [139.75, 35.65, 139.85, 35.70]
# 対象期間
start_time = "2024-08-01T00:00:00Z"
end_time = "2024-08-10T23:59:59Z"
# 使用する STAC カタログ
catalog_url = "https://earth-search.aws.element84.com/v1"
# 対象コレクション
collection_id = "sentinel-2-l1c"

required_bands = ["B01", "B02", "B04", "B05", "B08", "B8A", "B09", "B10", "B11", "B12"]
output_resolution = 60
output_crs = "EPSG:32654"

In [20]:
import folium
from shapely.geometry import box
import geojson
import datetime
import stackstac
import pystac_client
import numpy as np
import rasterio
from s2cloudless import S2PixelCloudDetector
import matplotlib.pyplot as plt

In [6]:
catalog = pystac_client.Client.open(catalog_url)

In [7]:
catalog

<Client id=earth-search-aws>

In [8]:
search = catalog.search(
    collections=[collection_id],
    bbox=bbox,
    datetime=f"{start_time}/{end_time}",
    query={"eo:cloud_cover": {"lt": 10}},
    # sortby=[{"field": "properties.eo:cloud_cover", "direction": "asc"}],
    limit=10
)


In [9]:
items = search.item_collection()
print(f"Found {len(items)} items.")

Found 1 items.


In [22]:
center_lon = (bbox[0] + bbox[2]) / 2
center_lat = (bbox[1] + bbox[3]) / 2

In [32]:
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles="CartoDB positron")

In [33]:
bbox_polygon = box(bbox[0], bbox[1], bbox[2], bbox[3])
bbox_geojson = geojson.Feature(geometry=bbox_polygon, properties={"name": "Search BBOX"})

folium.GeoJson(
    bbox_geojson,
    style_function=lambda x: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.1
    },
    tooltip="Search BBOX"
).add_to(m)

In [34]:
item_geometry_geojson = items[0].geometry # これは既に GeoJSON 形式の辞書

# STAC アイテムのジオメトリを地図に追加
folium.GeoJson(
    item_geometry_geojson,
    style_function=lambda x: {
        'fillColor': 'red',
        'color': 'red',
        'weight': 3,
        'fillOpacity': 0.2
    },
    tooltip=f"AOI: {items[0].id}"
).add_to(m)

In [35]:
item_bbox = items[0].bbox # [min_lon, min_lat, max_lon, max_lat]
m.fit_bounds([[item_bbox[1], item_bbox[0]], [item_bbox[3], item_bbox[2]]])

In [36]:
folium.LayerControl().add_to(m)

In [37]:
m

In [10]:
# https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1/collections/sentinel-2-l1c/items/S2B_54SUE_20240804_0_L1C?.language=ja
items[0]

<Item id=S2B_54SUE_20240804_0_L1C>

In [17]:
int(output_crs.split(":")[1])

32654

In [16]:
data_cube = stackstac.stack(
        items[0],
        assets=required_bands,
        resolution=output_resolution,
        epsg=int(output_crs.split(":")[1]), # stackstac は EPSG コードの数値部分を期待
        bounds_latlon=bbox, # BBox を指定してクリップ
        dtype="uint16", # L1C は通常 uint16
        rescale=False, # DN 値のまま読み込む
        # chunks={"x": 1024, "y": 1024, "band": -1} # メモリに合わせて調整
    )

AssertionError: out_bounds=None